In [ ]:
from utils import iteration_logging
from cycles.graph_cycle_iterator import GraphCycleIterator
from graph_edge_iterator import GraphEdgeIterator
from tqdm.auto import tqdm
import io
import pandas as pd

In [ ]:

OMEGA = 30
BUFFER_COUNT = 2
PRUNE_INTERVAL = 1_000
END_DATE = "2019-08-10"



In [ ]:
def edge_generator():
    return GraphEdgeIterator(end_date=END_DATE, buffer_count=BUFFER_COUNT)

def wrapped_edge_generator(log_stream):
    return tqdm(iteration_logging(edge_generator(), log_stream=log_stream), total=TOTAL_EDGES)

def cycle_generator(omega, log_stream_edges=None, track_history=False):
    return GraphCycleIterator(wrapped_edge_generator(log_stream_edges), omega, track_history=track_history)

In [ ]:
TOTAL_EDGES = len(list(edge_generator()))

In [ ]:
log_stream_edges = io.StringIO()
log_stream_cycles = io.StringIO()

wrapped_cycles = iteration_logging(cycle_generator(OMEGA, log_stream_edges), log_stream=log_stream_cycles)

cycles = [c for c in wrapped_cycles]


In [ ]:
dfs = []
for index, (prefix, log_stream) in enumerate({
    "_interactions": log_stream_edges,
    "cycles": log_stream_cycles
}.items()):
    log_stream.seek(0)
    df = pd.read_csv(log_stream)
    df["time_seconds"] = pd.to_datetime(df["time_seconds"], unit="s")
    if index > 0:
        df = df.drop(columns="memory_usage_bytes")
    df = df.set_index("time_seconds").add_prefix(prefix + "_")
    dfs.append(df)

df = pd.concat(dfs, axis=1)
df.plot(subplots=True)

In [ ]:
generator = cycle_generator(OMEGA, track_history=True)
list(generator)

In [ ]:
import numpy as np
import pandas as pd


def bin_lengths(lengths, bin_edges):
    """Bins length counts into predefined bins."""
    binned_counts = {f"{bin_edges[i]}-{bin_edges[i+1]}": 0 for i in range(len(bin_edges) - 1)}
    for length, count in lengths.items():
        for i in range(len(bin_edges) - 1):
            if bin_edges[i] <= length < bin_edges[i + 1]:
                binned_counts[f"{bin_edges[i]}-{bin_edges[i+1]}"] += count
                break
    return binned_counts

def process_data_for_plot(data):
    """Converts list of length-count dictionaries into a DataFrame for plotting."""
    all_lengths = set()
    for d in tqdm(data, "determining min/max"):
        all_lengths.update(d.keys())

    min_len, max_len = min(all_lengths), max(all_lengths)

    # Define sensible bins (logarithmic if range is large, otherwise equal-width)
    if max_len - min_len > 20:
        bin_edges = np.unique(np.logspace(np.log10(min_len), np.log10(max_len), num=10, dtype=int))
    else:
        bin_edges = np.arange(min_len, max_len + 5, step=max((max_len - min_len) // 10, 1))

    # Re-bin the data
    binned_data = [bin_lengths(d, bin_edges) for d in data]

    # Convert to DataFrame
    df = pd.DataFrame(binned_data).fillna(0)
    return df

def plot_stacked_area(df):
    """Plots a stacked area chart from the processed DataFrame."""
    df.plot(kind='area', stacked=True, colormap='cmap', alpha=0.7, figsize=(10, 6))
    plt.xlabel("Index")
    plt.ylabel("Count")
    plt.title("Stacked Area Chart of Binned Length Frequencies")
    plt.legend(title="Bins", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()


In [ ]:
# Process and plot
df = process_data_for_plot(generator.separate_seeds_history)
plot_stacked_area(df)

In [ ]:
# Process and plot
df = process_data_for_plot(generator.combined_seeds_history)
plot_stacked_area(df)

In [ ]:
minimums = list(cycle_generator(OMEGA))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(minimums)